# U.S. Medical Insurance Costs

### Import Data


In [2]:
import csv

insurance_data_dict = {}

with open('insurance.csv') as insurance_data_file:
    #Import data as a csv.DictReader object 
    insurance_data_dict_obj = csv.DictReader(insurance_data_file) 
    
    # Convert insurance_data_dict_obj to a python dictionary with an id number for each row
    row_num = 1
    for row in insurance_data_dict_obj:
        insurance_data_dict[row_num] = {field:data for (field, data) in zip(row, row.values())}
        row_num += 1

print(insurance_data_dict[1])

{'age': '19', 'sex': 'female', 'bmi': '27.9', 'children': '0', 'smoker': 'yes', 'region': 'southwest', 'charges': '16884.924'}


In [3]:
def convert_data_types(insurance_data_dict):
    # Converts all fields in insurance_data_dict to their correct numerical type
    # Fields changed: age, bmi, children, charges
    
    for id_num in insurance_data_dict:
        insurance_data_dict[id_num] = {
            'age': int(insurance_data_dict[id_num]['age']), 
            'sex': insurance_data_dict[id_num]['sex'], 
            'bmi': float(insurance_data_dict[id_num]['bmi']), 
            'children': int(insurance_data_dict[id_num]['children']), 
            'smoker': insurance_data_dict[id_num]['smoker'], 
            'region': insurance_data_dict[id_num]['region'], 
            'charges': float(insurance_data_dict[id_num]['charges'])}

convert_data_types(insurance_data_dict)
print(insurance_data_dict[1])

{'age': 19, 'sex': 'female', 'bmi': 27.9, 'children': 0, 'smoker': 'yes', 'region': 'southwest', 'charges': 16884.924}


### Suggested Analysis

* **Average age of patients**

In [4]:
num_of_patients = len(insurance_data_dict)
sum_of_ages = sum([insurance_data_dict[id_num]['age'] for id_num in insurance_data_dict])

avg_age_of_patient = round(sum_of_ages / num_of_patients, 2)
avg_age_of_patient

39.21

* **Location Breakdown**

In [5]:
regions, region_counts = [], []

for id_num in insurance_data_dict:
    region = insurance_data_dict[id_num]['region']
    if region not in regions:
        regions.append(region)
        region_counts.append(1)
    else:
        region_counts[regions.index(region)] += 1

region_percents = [round(100 * count / num_of_patients, 2) for count in region_counts]

region_dict_name = ['Count', 'Percent']

region_dict = {region:{field:val for (field, val) in zip(region_dict_name, region_num_zip)} 
               for (region, region_num_zip) in zip(regions, zip(region_counts, region_percents))}
region_dict

{'southwest': {'Count': 325, 'Percent': 24.29},
 'southeast': {'Count': 364, 'Percent': 27.2},
 'northwest': {'Count': 325, 'Percent': 24.29},
 'northeast': {'Count': 324, 'Percent': 24.22}}

* **Smoker Cost Breakdown**

In [6]:
smoker_count, smoker_cost_total = 0, 0

for id_num in insurance_data_dict:
    smoker = insurance_data_dict[id_num]['smoker']
    if smoker == 'yes':
        smoker_count += 1
        smoker_cost_total += insurance_data_dict[id_num]['charges']

total_cost = sum([insurance_data_dict[id_num + 1]['charges'] for id_num in range(num_of_patients)])

smoker_dict = {'Smokers':
               {'Count':smoker_count, 
                'Count_Percentage':round(100 * smoker_count/num_of_patients,2), 
                'Total Costs':round(smoker_cost_total,2), 
                'Cost_Percentage':round(100 * smoker_cost_total / total_cost,2),
                'Average_cost':round(smoker_cost_total / smoker_count, 2)},
               'Non-smokers':
               {'Count':num_of_patients - smoker_count, 
                'Count_Percentage':round(100 * (num_of_patients - smoker_count)/num_of_patients,2), 
                'Total Costs':round(total_cost - smoker_cost_total,2), 
                'Cost_Percentage':round(100 * (total_cost - smoker_cost_total) / total_cost,2),
                'Average_cost':round((total_cost - smoker_cost_total) / (num_of_patients - smoker_count), 2)}}

print(smoker_dict)
print('Smokers average costs are {} times greater than Non-smokers average costs!'.format(
    round(smoker_dict['Smokers']['Average_cost'] / smoker_dict['Non-smokers']['Average_cost'], 2)))

{'Smokers': {'Count': 274, 'Count_Percentage': 20.48, 'Total Costs': 8781763.52, 'Cost_Percentage': 49.46, 'Average_cost': 32050.23}, 'Non-smokers': {'Count': 1064, 'Count_Percentage': 79.52, 'Total Costs': 8974061.47, 'Cost_Percentage': 50.54, 'Average_cost': 8434.27}}
Smokers average costs are 3.8 times greater than Non-smokers average costs!


* **Average age for parents**

In [84]:
parent_ages = [insurance_data_dict[id_num]['age'] for id_num in insurance_data_dict if insurance_data_dict[id_num]['children'] > 0]
avg_parent_age = round(sum(parent_ages) / len(parent_ages), 2)
print("The average age of a parent patients is {} years old.".format(avg_parent_age))

The average age of a parent patients is 39.78 years old.


### Extended Analysis
#### Linear Regression for Cost estimate
Use an extended verion of the method in the Reggie_Linear_Regression project to create an insurance cost estimate function.

In [89]:
def insurance_data_to_list(insurance_data_dict):
    # Convert the insurance_data_dict to a list of lists with numeric values for all fields.
    # Note: sex:'female'=0,'male'=1,  smoker:'no'=0,'yes'=1,  'region':'southwest'=0,'southeast'=1,'northwest'=2,'northeast'=3
    return [[
        patient_dict['age'],
        ['female','male'].index(patient_dict['sex']),
        patient_dict['bmi'],
        patient_dict['children'],
        ['no','yes'].index(patient_dict['smoker']),
        ['southwest','southeast','northwest','northeast'].index(patient_dict['region']),
        patient_dict['charges']] for patient_dict in insurance_data_dict.values()]

# Convert the insurance_data_dict to a data list
insurance_data_list = insurance_data_to_list(insurance_data_dict)

def get_cost(factors, patient_data):
    # returns cost = age_factor * age + sex_factor * sex + ... + region_factor * region + intercept
    # Note: sex:'female'=0,'male'=1,  smoker:'no'=0,'yes'=1,  'region':'southwest'=0,'southeast'=1,'northwest'=2,'northeast'=3
    return sum([factor * p_data for (factor, p_data) in zip(factors[:-1], patient_data[:-1])]) + factors[-1]

# Produce an example of the estimated insurance cost for the first patient in insurance_data_list.
example_cost_estimate = get_cost([1,1,1,1,1,1,1],insurance_data_list[0])
print("Example of cost estimate for first patient: $" + str(example_cost_estimate))

Example of cost estimate for first patient: $48.9


In [94]:
def calculate_error(factors, patient_data):
    # Calculate the absolute different between the estimated cost 
    # produced from get_cost and the true cost from the data.
    cost = patient_data[-1]   
    return abs(get_cost(factors, patient_data) - cost)

# Produce an example of the error of the estimated insurance cost for the first patient in insurance_data_list
example_cost_error = calculate_error([1,1,1,1,1,1,1],insurance_data_list[0])

# Get the actual insurance cost of the first patient
first_patient_real_cost = insurance_data_list[0][-1]

print("Actual Cost of first patient: $" + str(first_patient_real_cost))
print("Difference between actual cost and example estimate: $" + str(example_cost_error))
print(first_patient_real_cost == example_cost_error + example_cost_estimate)

Actual Cost of first patient: $16884.924
Difference between actual cost and example estimate: $16836.023999999998
True


In [95]:
def calculate_all_error(factors, insurance_data_list):
    # Given a single set of factors, return the total error between 
    # all of the estimated costs and all of the acutal costs 
    return sum([calculate_error(factors, patient_data) for patient_data in insurance_data_list])

example_total_cost_error = calculate_all_error([1,1,1,1,1,1,1], insurance_data_list)
print("Total error of example estimates: $" + str(example_total_cost_error))

Total error of example estimates: $17656599.365758996


### Note:
When preforming the regression I found it very difficult to give a good guess of the optimal factors. Due to the exponential nature of the total number of combinations I couldn't, for example, give each factor 20 values. This is because there would be 20^7=1,280,000,000 factor combinations where each factor combination would be used to estimate costs for 1338 patients.<br>
<br>
My approach was as follows:
1. Start with the random set of factors called **current_best**=[1,1,1,1,1,1,1] and a **smallest_error** = infinity
2. Choose a large **factor_offset** of 100
3. Create the **possible_factors** list which contains lists of three new possible factors: [1 - offset, 1, 1 + offset]
    * [[-99, 1, 101], [-99, 1, 101], [-99, 1, 101], [-99, 1, 101], [-99, 1, 101], [-99, 1, 101], [-99, 1, 101]]
<br><br>
4. Loop through all the different combinations of the possible_factors list. This is only 3^7=2187 combinations which is much more manageable. [-99,-99,-99,-99,-99,-99,-99], [-99,-99,-99,-99,-99,-99,1], [-99,-99,-99,-99,-99,-99,101], ..., [1,1,1,1,1,1,1], [1,1,1,1,1,1,101], ... [101,101,101,101,101,101,-99], [101,101,101,101,101,101,1], and [101,101,101,101,101,101,101]
<br><br>
5. Durring the loop find the combination of factors that produces the lowest total error. Set it to **current_best** and set **smallest_error** equal to the total error for that combination.
6. Add that combination and total error into the list **best_factors_sets**
<br><br>
7. Shift each list in **possible_factors** up or down by the **factor_offset** or not at all by comparing it to the old **current_best**. 
    * For example, suppose the new best combination is [101,-99,1,101,101,101,-99]. 
        * Since the first factor in our new list(101) is greater than the first factor in the old **current_best**([1,1,1,1,1,1,1]) we shift the first list in **possible_factors** up by the offset: [-99 + offset, 1 + offset, 101 + offset]=[1,101,201].
        <br><br>
        * Since the second factor in our new list(-99) is less than the second factor in the old **current_best**([1,1,1,1,1,1,1]) we shift the second list in **possible_factors** down by the offset: [-99 - offset, 1 - offset, 101 - offset]=[-199,-99,1].
        <br><br>
        * Since the third factor in our new list(1) is the same as the third factor in the old **current_best**([1,1,1,1,1,1,1]) we don't shift the third list in **possible_factors** so it stays as: [-99, 1, 101].
    * Thus **possible_factors** becomes [[1,101,201], [-199,-99,1], [-99, 1, 101], [1,101,201], [1,101,201], [1,101,201], [-199,-99,1]]
<br><br>
8. Repeat steps 4-7 for a given **number_of_rounds**

Eventually you will see the **current_best** factors in **best_factors_sets** will stop changing. Lower the **factor_offset** and continue the regression. After getting down to a **factor_offset** of 1, my best set of factors was [264, -453, 34, 390, 31257, 218, -4901] with a total error of about $4,561,918.03.
***

In [100]:
best_factors_sets = [[264, -453, 34, 390, 31257, 218, -4901, 4561918.027399002]]

In [102]:
# Set initial variables
number_of_rounds = 10
smallest_error = float('inf')
best_factors = [1, 1, 1, 1, 1, 1, 1] 
factor_offset = 1

# Set the current best set of factors to the last set in best_factors_sets
# cbf + 0 is used to avoid aliasing 
current_best = [cbf + 0 for cbf in best_factors_sets[-1]]

# Set the possible_factors list as described in Step 3. above
possible_factors = [[cb - factor_offset, cb, cb + factor_offset] for cb in current_best]

# Repeats Step4-Step7 number_of_rounds times.
for i in range(number_of_rounds):
    # Prints the percentage of progress made
    print(str(round(100 * i / number_of_rounds,2)) + '%')
    
    # Step 4-Step5
    for m1 in possible_factors[0]:
        for m2 in possible_factors[1]:
            for m3 in possible_factors[2]:
                for m4 in possible_factors[3]:
                    for m5 in possible_factors[4]:
                        for m6 in possible_factors[5]:
                            for m7 in possible_factors[6]:
                                total_error = calculate_all_error([m1,m2,m3,m4,m5,m6,m7], insurance_data_list)
                                if total_error < smallest_error:
                                    smallest_error = total_error
                                    best_factors = [m1,m2,m3,m4,m5,m6,m7]
    
    # Step 6
    best_factors_sets.append(best_factors + [smallest_error])
    
    # Step 7
    for factor_index in range(len(best_factors)):
        if best_factors[factor_index] == possible_factors[factor_index][0]:
            possible_factors[factor_index][0] -= factor_offset
            possible_factors[factor_index][1] -= factor_offset
            possible_factors[factor_index][2] -= factor_offset
        elif best_factors[factor_index] == possible_factors[factor_index][-1]:
            possible_factors[factor_index][0] += factor_offset
            possible_factors[factor_index][1] += factor_offset
            possible_factors[factor_index][2] += factor_offset
    
# Prints the last 10 best factor sets
best_factors_sets[-10:]

0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%


[[264, -453, 34, 390, 31257, 218, -4901, 4561918.027399002],
 [264, -453, 34, 390, 31257, 218, -4901, 4561918.027399002],
 [264, -453, 34, 390, 31257, 218, -4901, 4561918.027399002],
 [264, -453, 34, 390, 31257, 218, -4901, 4561918.027399002],
 [264, -453, 34, 390, 31257, 218, -4901, 4561918.027399002],
 [264, -453, 34, 390, 31257, 218, -4901, 4561918.027399002],
 [264, -453, 34, 390, 31257, 218, -4901, 4561918.027399002],
 [264, -453, 34, 390, 31257, 218, -4901, 4561918.027399002],
 [264, -453, 34, 390, 31257, 218, -4901, 4561918.027399002],
 [264, -453, 34, 390, 31257, 218, -4901, 4561918.027399002]]

In [109]:
# Best set of factors [264, -453, 34, 390, 31257, 218, -4901]
lowest_total_error = calculate_all_error([264, -453, 34, 390, 31257, 218, -4901], insurance_data_list)
avg_cost_error = lowest_total_error / len(insurance_data_list)

print('The lowest total insurance cost error is: $' + str(lowest_total_error))
print('The average error of the cost estimate is: $' + str(avg_cost_error))

The lowest total insurance cost error is: $4561918.027399002
The average error of the cost estimate is: $3409.505252166668


### Note 2: 

I tried to use `itertools` to remove the seven nested for loops in the regression, but I had to modify step 7. <br><br>
Since the nested loops are directly linked to the **possible_factors** list, when I update the **possible_factors** list the factor combinations created by the nested loops are also automatically updated.<br><br>
It turns out that when **possible_factors_iter** is updated the **possible_factor_tuples** list doesn't automatically update. This means I needed to recreate the **possible_factor_tuples** each time Steps4-7 are repeated.    

In [54]:
import itertools

In [131]:
best_factors_sets_iter = [(261, -459, 41, 401, 31201, 241, -5029, 4562520.660198994)]

In [134]:
# Set initial variables
number_of_rounds_iter = 10
smallest_error_iter = float('inf')
best_factors_iter = [1, 1, 1, 1, 1, 1, 1] 
factor_offset_iter = 10

# Set the current best set of factors to the last set in best_factors_sets
# cbf + 0 is used to avoid aliasing 
current_best_iter = [cbf + 0 for cbf in best_factors_sets_iter[-1]]

# Set the possible_factors list as described in Step 3. above
possible_factors_iter = [[cb - factor_offset_iter, cb, cb + factor_offset_iter] for cb in current_best_iter]

# Repeats Step4-Step7 number_of_rounds times.
for i in range(number_of_rounds_iter):
    # Prints the percentage of progress made
    print(str(round(100 * i / number_of_rounds,2)) + '%')
    
    # Step 4-Step5
    possible_factor_tuples = list(itertools.product(possible_factors_iter[0], possible_factors_iter[1], 
                                                    possible_factors_iter[2], possible_factors_iter[3], 
                                                    possible_factors_iter[4], possible_factors_iter[5], 
                                                    possible_factors_iter[6]))
    for factor_tuple in possible_factor_tuples:
        total_error = calculate_all_error(factor_tuple, insurance_data_list)
        if total_error < smallest_error_iter:
            smallest_error_iter = total_error
            best_factors_iter = factor_tuple
    
    # Step 6
    best_factors_sets_iter.append(best_factors_iter + (smallest_error_iter,))
    
    # Step 7
    for factor_index in range(len(best_factors_iter)):
        if best_factors_iter[factor_index] == possible_factors_iter[factor_index][0]:
            possible_factors_iter[factor_index][0] -= factor_offset_iter
            possible_factors_iter[factor_index][1] -= factor_offset_iter
            possible_factors_iter[factor_index][2] -= factor_offset_iter
        elif best_factors_iter[factor_index] == possible_factors_iter[factor_index][-1]:
            possible_factors_iter[factor_index][0] += factor_offset_iter
            possible_factors_iter[factor_index][1] += factor_offset_iter
            possible_factors_iter[factor_index][2] += factor_offset_iter
    
# Prints the last 10 best factor sets
best_factors_sets_iter[-10:]

0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%


[(261, -409, 41, 441, 31191, 241, -5059, 4565880.171798993),
 (261, -419, 41, 431, 31201, 231, -5069, 4563700.579998995),
 (261, -429, 41, 421, 31201, 231, -5059, 4563087.478798995),
 (261, -439, 41, 411, 31191, 241, -5049, 4562683.076098993),
 (261, -449, 41, 401, 31201, 241, -5039, 4562567.403998993),
 (261, -459, 41, 401, 31201, 241, -5029, 4562520.660198994),
 (261, -459, 41, 401, 31201, 241, -5029, 4562520.660198994),
 (261, -459, 41, 401, 31201, 241, -5029, 4562520.660198994),
 (261, -459, 41, 401, 31201, 241, -5029, 4562520.660198994),
 (261, -459, 41, 401, 31201, 241, -5029, 4562520.660198994)]